#### Use the TDA to instead of community detection and perform hubs detection

In [5]:
import subprocess
import igraph as ig
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
plt.rcParams['svg.fonttype'] = 'none'

Import GRN

In [6]:
# Step 1: Read the content from the local file
with open('./mapper_nodes.txt', 'r') as file:
    content = file.read()

# Step 2: Interpret the content with 'eval' or 'ast.literal_eval'
# Create a dictionary to safely handle 'array'
data = eval(content, {"array": np.array})

# Step 3: Convert data to a DataFrame for easier manipulation
rows = []
for value, genes in data:
    for gene in genes:
        rows.append({'Value': value, 'Gene': gene})

df = pd.DataFrame(rows)

In [7]:
df.to_csv('Mapper_node_df.tsv', sep='\t', index=False)

In [8]:
cGRN = pd.read_csv('./Data/Curated_gene_regulatory_network.tsv', sep='\t')
mapper_cluster = df

In [9]:
# Create an igraph Graph from the edge list
graph = ig.Graph.TupleList(cGRN[['TF', 'Target_Gene']].itertuples(index=False), directed=True)

# Print summary of the graph
print("Summary of the graph:")
print("Number of vertices:", graph.vcount())
print("Number of edges:", graph.ecount())

Summary of the graph:
Number of vertices: 1786
Number of edges: 4201


In [10]:
node_names = graph.vs['name']  # Get the names of the vertices
in_degrees = graph.degree(mode='in')  # In-degree
out_degrees = graph.degree(mode='out')  # Out-degree
total_degrees = graph.degree(mode='all')  # Total degree

degree_df = pd.DataFrame({
    'Gene_Name': node_names,
    'In_Degree': in_degrees,
    'Out_Degree': out_degrees,
    'Total_Degree': total_degrees
})

In [11]:
degree_df.to_csv('Degree_CGRN.tsv', sep='\t', index=False)

*********

### Hub by grouping similar cluster

Thanks to the adjacency plots, we identified 4 mega clusters defined as :
- 1 - 0+2+3+5 (multiple shared)
- 2 - 7+19 (spe shared)
- 3 - 18+9+13+17 (multiple exclusif)
- 4 - 11 (spe exclusif)

We can identifed hubs on these mega clusters

In [12]:
custom_cluster = {1: [0,2,3,5],
                  2: [7],
                  3: [9,18,13,15,17,11],
                  4: [19]}
# Dictionary to store community and corresponding hub nodes
community_hubs = {}
# Dictionary to store centrality values for each node
node_centrality = {}
edge_counts = {}
nodes_counts = {}
## Save the TFs in each mega cluster
TF_in_mega_cluster_saved = {}
# Iterate through communities
for community_id in custom_cluster:
    
    TF_in_mega_cluster = mapper_cluster[mapper_cluster['Value'].isin(custom_cluster[community_id])]['Gene'].unique() # Find unique TF in the mega cluster
    TF_in_mega_cluster_saved[community_id] = TF_in_mega_cluster
    sub_cGRN = cGRN[(cGRN['TF'].isin(TF_in_mega_cluster))] # Retrieve the subGRN composed of the TF in the mega cluster
    subgraph = ig.Graph.TupleList(sub_cGRN[['TF', 'Target_Gene']].itertuples(index=False), directed=True) #create the subgraph
    nodes_counts[community_id] = subgraph.vcount()
    edge_counts[community_id] = subgraph.ecount()

    # Compute centrality measures for nodes in the community subgraph
    degree_centrality = subgraph.degree()
    betweenness_centrality = subgraph.betweenness(directed=True)

    # Find top two nodes for degree centrality
    degree_centrality_with_names = [(subgraph.vs[node]["name"], centrality) for node, centrality in enumerate(degree_centrality)]
    degree_centrality_with_names.sort(key=lambda x: x[1], reverse=True)

    max_degree_nodes = degree_centrality_with_names[:1]  # Top 2 nodes
    max_degree_nodes = [node[0] for node in max_degree_nodes]  # Extract names

    # Find top two nodes for betweenness centrality
    betweenness_centrality_with_names = [(subgraph.vs[node]["name"], centrality) for node, centrality in enumerate(betweenness_centrality)]
    betweenness_centrality_with_names.sort(key=lambda x: x[1], reverse=True)

    max_betweenness_nodes = betweenness_centrality_with_names[:1]  # Top 2 nodes
    max_betweenness_nodes = [node[0] for node in max_betweenness_nodes if node[1] > 0]  # Exclude zero centrality

    # Combine all nodes with the highest centrality values
    hub_nodes = list(set(max_degree_nodes + max_betweenness_nodes))

    # Store the hub nodes for the community
    community_hubs[community_id] = hub_nodes

    # Update node_centrality dictionary with centrality values for nodes in the community
    for node_index, node_name in enumerate(subgraph.vs["name"]):
        node_centrality.setdefault(node_name, {}).update({
            "Degree Centrality": degree_centrality[node_index],
            "Betweenness Centrality": betweenness_centrality[node_index],
        })

# Print the dictionary of community and corresponding hub nodes
print("Community - Hub Nodes:")
for community, hub_nodes in community_hubs.items():
    print(f"Community {community}: Hubs {hub_nodes}")

Community - Hub Nodes:
Community 1: Hubs ['Solyc07g054220', 'Solyc06g051840']
Community 2: Hubs ['Solyc03g093550']
Community 3: Hubs ['Solyc07g063420', 'Solyc06g082430']
Community 4: Hubs ['Solyc10g047640', 'Solyc12g009240']


In [13]:
df = pd.DataFrame([(key, value) for key, values in TF_in_mega_cluster_saved.items() for value in values],
                  columns=['Mega_Cluster', 'TF'])

In [14]:
# df.to_csv('TF_in_mega_cluster.tsv', sep='\t', index=False)

Compare with activity

In [15]:
def expand_dataframe(community_dict):
    expanded_rows = []
    
    for gene, communities in community_dict.items():
        for community in communities:
            expanded_rows.append([gene, community])  # Append as a list of values
    
    return pd.DataFrame(expanded_rows, columns=['Cluster', 'OLN'])

In [16]:
res_hubs = pd.read_csv('../Results_hubs_activity_stats.tsv',sep='\t')

In [17]:
pd.set_option('display.max_colwidth', None)

In [18]:
Hubs_mega_cluster = [TF for TFs in community_hubs.values() for TF in TFs]

In [19]:
res_mega_cluster = res_hubs[res_hubs['OLN'].isin(Hubs_mega_cluster)][['OLN','Groups', 'gene.description', 'gene.family']]

In [20]:
res_mega_cluster[~res_mega_cluster['Groups'].isna()]['OLN'].to_list()

['Solyc03g093550',
 'Solyc06g051840',
 'Solyc07g054220',
 'Solyc07g063420',
 'Solyc12g009240']

In [21]:
expanded_df = expand_dataframe(community_hubs)

In [22]:
res_comm = pd.merge(res_mega_cluster,expanded_df, on='OLN', how='right')

In [23]:
res_comm = res_comm[~res_comm['Groups'].isna()]

In [24]:
res_comm.rename(columns={'Cluster':'Mega_Cluster'}, inplace=True)

In [25]:
res_comm

,OLN,Groups,gene.description,gene.family,Mega_Cluster
0,Solyc07g054220,PSTVd_S23_pval & Bcinerea_pval & Mincognita_14dpi_pval,Ethylene-responsive transcription factor (AHRD V3.3 *** A0A2G3C0R2_CAPCH),ERF,1
1,Solyc06g051840,Cfulvum_pval,Solanum lycopersicum Cytokinin Response Factor 6,ERF,1
2,Solyc03g093550,Bcinerea_pval & Mincognita_14dpi_pval,Ethylene-responsive transcription factor 5 (AHRD V3.3 *** A0A2G3CVP7_CAPCH),ERF,2
3,Solyc07g063420,Cfulvum_pval & Bcinerea_pval,NAC domain-containing protein (AHRD V3.3 *** A0A2U1LQQ4_ARTAN),NAC,3
6,Solyc12g009240,Bcinerea_pval,Ethylene-responsive transcription factor (AHRD V3.3 *-* A0A2G3BP26_CAPCH),ERF,4


In [26]:
res_comm.to_csv('./Results_hubs_by_TDA_mega_cluster.tsv', sep='\t', index=False)